# Inference using the SLAP models 

Predict reaction outcome for a single reaction or batch of reactions.

The input to this are atom-mapped reactionSMILES. 
-> note: Ideally we would just put in the target molecule and construct the one or two reactions leading to it.

They do not need to be balanced. 
They can be supplied directly as a txt file with one reactionSMILES per line

In [23]:
import pathlib
import sys
sys.path.append(str(pathlib.Path("__file__").absolute().parents[1]))

from src.model.classifier import load_trained_model
from src.data.dataloader import SLAPDataset, collate_fn

from torch.utils.data import DataLoader
import pytorch_lightning as pl

In [2]:
# load data
# CHANGE raw_dir and name to the location of your file
# expects a file with "SMILES" in the first line and one reactionSMILES on each following line
data = SLAPDataset(
            name="reactions.txt",
            raw_dir="../data/",
            reaction=True,
            graph_type="bond_nodes",
            featurizers="custom",
            label_column=None,
)

Done saving data into cached files.


In [24]:
# paths to the best models
model_0D = "tba"
model_1D = "tba"
model_2D = "../production_models/2022-12-06-115456_273019/best.ckpt"

In [28]:
# load the trained model if it is not loaded
if isinstance(model_2D, str):
    model_2D = load_trained_model("D-MPNN", model_2D)
    model_2D.eval()
trainer = pl.Trainer(accelerator="gpu", logger=False, max_epochs=-1)
# apply it
dl = DataLoader(data, collate_fn=collate_fn)
trainer.predict(model_2D, dl)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/julian/miniconda3/envs/slap-gnn/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

[tensor([0.9082]),
 tensor([0.5955]),
 tensor([0.8921]),
 tensor([0.4336]),
 tensor([0.2668]),
 tensor([0.2839]),
 tensor([0.0928]),
 tensor([0.8578]),
 tensor([0.8729])]